<a href="https://colab.research.google.com/github/xtbtds/ml-zoomcamp/blob/main/lesson5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.1 Intro

***Deployment of the model*** - use the model to predict new values without running the code. The way to do this is to deploy the model in a server (run the code and make the model). After deploying the code in a machine used as server we can make some **endpoints** (using api's) to connect from another machine to the server and predict values.



# 5.2 Saving and loading the model

- To save the model we made before there is an option using the **pickle library**:
  - First install the library with the command `pip install pickle-mixin` if you don't have it.
  - After training the model and being the model ready for prediction process use this code to save the model for later.
  ``` 
  import pickle
with open('model.bin', 'wb') as f_out:
   pickle.dump((dcit_vectorizer, model), f_out)
f_out.close() ## After opening any file it's nessecery to close it
  ```
  - In the code above we'll making a **binary file named model.bin** and writing the dict_vectorizer for one hot encoding and model as array in it. (We will save it as binary in case it wouldn't be readable by humans)
  - To be able to use the model in future without running the code, We need to open the binary file we saved before.
  ```
with open('mode.bin', 'rb') as f_in:  ## never open a binary file you do not trust!
    dict_vectorizer, model = pickle.load(f_in)
f_in.close()
  ```
  - With unpacking the model and the dict_vectorizer, We're able to again predict for new input values without training a new model by re-running the code.

# 5.3 Web services: introduction to Flask

A web service is a method used to communicate between electronic devices.

- Methods:
  - **GET**: a method used to retrieve files, for example when we are searching for a cat image in google we are actually requesting cat images with GET method.
  - **POST**: for example in a sign up process, when we are submiting our name, username, passwords, etc we are posting our data to a server that is using the web service (there is no specification where the data goes).
  - **PUT**: same as POST but we are specifying where the data is going to.
  - **DELETE**: a method that is used to request to delete some data from the server.

- 0.0.0.0 vs localhost
  - process that is listening on 127.0.0.1 for connections will only receive *local* connections on that socket.
  - server is told to listen on 0.0.0.0 will "listen on every available network interface"

- Create Flask web service
  - `pip install Flask`
  -
  ```
  from flask import Flask
app = Flask('churn-app')
@app.route('/ping',methods=[GET])
def ping():
      return 'PONG'
if __name__=='__main__':
      app.run('debug=True, host='0.0.0.0', port=9696)
  ```
  - open your browser and search localhost:9696/ping or type `curl 0.0.0.0:9696/ping`

# 5.4 Serving the churn model with Flask

- First, load the previous saved model and use a prediction function in a special route.
```
import pickle
with open('churn-model.bin', 'rb') as f_in:
      dv, model = pickle.load(f_in)
  ```
- Predict a value for a customer
```
def predict_single(customer, dv, model):
  X = dv.transform([customer])  ## apply the one-hot encoding feature to the customer data 
  y_pred = model.predict_proba(X)[:, 1]
  return y_pred[0]
```
- Make the final function used for creating the web service.
```
@app.route('/predict', methods=['POST'])  
## send the customer information = POST
def predict():
    customer = request.get_json()  ## access the body of json.
    prediction = predict_single(customer, dv, model)
    churn = prediction >= 0.5
    result = {
        'churn_probability': float(prediction), ## conver numpy data into python data in flask framework
        'churn': bool(churn),  ## conver
    }
    return jsonify(result)  ## send back the data in json format to the user
```
- To see the result can't use a simple request in web browser. We can run the code below to post a new user data and see the response. Browser works corretly with GET requests, but we have POST.
```
## a new customer informations
customer = {
  'customerid': '8879-zkjof',
  'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'tenure': 41,
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'no',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'bank_transfer_(automatic)',
  'monthlycharges': 79.85,
  'totalcharges': 3320.75
}
```
```
import requests ## to use the POST method 
url = 'http://localhost:9696/predict' ## the route for prediction
response = requests.post(url, json=customer) ## post the customer information in json format
result = response.json() ## get the server response
print(result)
```
-  When you run your app you will see a warning that it is not a **WGSI server and not suitable for production environmnets**.
  - **gunicorn** - for Linux and Mac OS

    - install: `pip install gunicorn`
    - run: `gunicorn --bind 0.0.0.0:9696 churn:app`
    - in churn:app the name churn is the name we set for our the file containing the code app = Flask('churn')(for example: churn.py)
  - **waitress** - for Windows
    - install: `pip install waitress`
    - run: `waitress-serve --listen=0.0.0.0:9696 churn:app`



# 5.5 Python virtual environment: Pipenv

- Every time we're running a file from a directory we're using the executive files from a global directory. For example when we install python on our machine the executable files that are able to run our codes will go to somewhere like /home/username/python/bin/ for example the pip command may go to /home/username/python/bin/pip.
- Sometimes the versions of libraries conflict (the project may not run or get into massive errors). For example we have an old project that uses sklearn library with the version of 0.24.1 and now we want to run it using sklearn version 1.0.0. We may get into errors because of the version conflict.
- To solve the conflict we can make virtual environments. Virtual environment is something that can seperate the libraries installed in our system and the libraries with specified version we want our project to run with. 
- **pipenv**
  - install: `pip install pipenv`
  - install the libraries we want for our project in the new virtual environment: `pipenv install numpy sklearn==0.24.1 flask`
  - using the pipenv command we made two files Pipfile and Pipfile.lock
  - If we want to run the project in another machine, we run `pipenv install`. This command will look into Pipfile and Pipfile.lock to install the libraries with specified version.
  - run the project in the virtual environment: `pipenv shell`

# 5.6 Environment management: Docker

To isolate more our project file from our system machine. With Docker you are able to pack all your project is a system that you want and run it in any system machine.   
- **Docker**
  - `sudo apt-get install docker.io`
- Once our project was packed in a Docker container, we're able to run our project on any machine.
- First we have to make a Docker image. In Docker image file there are settings and dependecies we have in our project. To find Docker images that you need you can simply search the Docker website.
- Create **Docker file**
    ```
    # First install the python 3.8, the slim version have less size
    FROM python:3.8.12-slim
    ```
    ```
    # Install pipenv library in Docker 
    RUN pip install pipenv
    ```
    ```
    # we have created a directory in Docker named app and we're using it as work directory 
    WORKDIR /app    
    ```
    ```                                                            
    # Copy the Pip files into our working derectory 
    COPY ["Pipfile", "Pipfile.lock", "./"]
    ```
    ```
    # install the pipenv dependecies we had from the project and deploy them 
    RUN pipenv install --deploy --system
    ```
    ```
    # Copy any python files and the model we had to the working directory of Docker 
    COPY ["*.py", "churn-model.bin", "./"]
    ```
    ```
    # We need to expose the 9696 port because we're not able to communicate with Docker outside it
    EXPOSE 9696
    ```
    ```
    # If we run the Docker image, we want our churn app to be running
    ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_serving:app"]
    ```
    - If we don't put the last line ENTRYPOINT, we will be in a python shell. Note that for the entrypoint, we put our commands in doble quotes.
- Build:
`docker build -t churn-prediction`
- Run: `docker run -it -p 9696:9696 churn-prediction:latest`
  - first 9696 is the port number of our machine and the last one is Docker container port